In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [60]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    next_states_logits, Qs_logits = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                        action_size=action_size, state_size=state_size)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=next_states_logits,
                                                                   labels=next_states_labels))
    #################################################### s'', Q' = env(s', a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    next_next_states_logits, nextQs_logits = Env(actions=next_actions_logits, states=next_states, # given
                                                 hidden_size=hidden_size, action_size=action_size, 
                                                 state_size=state_size, reuse=True)
    #################################################### s'', Q' = env(~s', ~a')
    next_actions_logits_ = Act(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
                               reuse=True)
    next_next_states_logits_, nextQs_logits_ = Env(actions=next_actions_logits_, states=next_states_logits, # pred
                                                   hidden_size=hidden_size, action_size=action_size, 
                                                   state_size=state_size, reuse=True)
    ##################################################### GAN
    Qs = tf.reshape(Qs_logits, shape=[-1])
    nextQs = tf.reshape(nextQs_logits, shape=[-1]) * (1-dones)
    nextQs_ = tf.reshape(nextQs_logits_, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Qs, # GAN
                                                                    labels=rates)) # 0-1 real
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(nextQs+nextQs_)/2, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(nextQs+nextQs_)/2, # GAN
                                                                    labels=tf.ones_like(rates))) # max
    ########################################################## Qlearning
    targetQs = rewards + (nextQs+nextQs_)/2
    eloss += tf.reduce_mean(tf.square(Qs - targetQs))
    ######################################################### Curosity=intrinsic reward
#     rloss = tf.square(Qs2-Qs2_)
#     sloss = tf.square(next_actions_logits-next_states_logits_)
#     sloss = tf.reduce_sum(, 1)
#     #### Q/r loss
#     mean, var = tf.nn.moments(x=rloss,axes=[0])
#     std = tf.sqrt(var)
#     rloss_norm = (rloss - mean)/ std 
#     #### State loss
#     mean, var = tf.nn.moments(x=sloss,axes=[0])
#     std = tf.sqrt(var)
#     sloss_norm = (sloss - mean)/ std
#     #### Intrinsic_rewards
#     intrinsic_rewards = sloss_norm + rloss_norm
    return actions_logits, aloss, eloss, aloss2

In [61]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [62]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [63]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [64]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [65]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [66]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [67]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:17.0000 R:17.0000 rate:0.0340 aloss:0.7135 eloss:3.0977 aloss2:0.7010 exploreP:0.9983
Episode:1 meanR:31.5000 R:46.0000 rate:0.0920 aloss:0.7179 eloss:3.0939 aloss2:0.7042 exploreP:0.9938
Episode:2 meanR:26.0000 R:15.0000 rate:0.0300 aloss:0.7127 eloss:3.0582 aloss2:0.7118 exploreP:0.9923
Episode:3 meanR:22.7500 R:13.0000 rate:0.0260 aloss:0.7208 eloss:3.0838 aloss2:0.7127 exploreP:0.9910
Episode:4 meanR:23.6000 R:27.0000 rate:0.0540 aloss:0.7204 eloss:3.0828 aloss2:0.7064 exploreP:0.9884
Episode:5 meanR:26.1667 R:39.0000 rate:0.0780 aloss:0.7257 eloss:3.0692 aloss2:0.7054 exploreP:0.9846
Episode:6 meanR:25.2857 R:20.0000 rate:0.0400 aloss:0.7113 eloss:3.0724 aloss2:0.7075 exploreP:0.9826
Episode:7 meanR:23.5000 R:11.0000 rate:0.0220 aloss:0.7117 eloss:3.0706 aloss2:0.7051 exploreP:0.9816
Episode:8 meanR:23.4444 R:23.0000 rate:0.0460 aloss:0.7160 eloss:3.0688 aloss2:0.7075 exploreP:0.9793
Episode:9 meanR:23.5000 R:24.0000 rate:0.0480 aloss:0.7032 eloss:3.0553 aloss2:0.7

Episode:80 meanR:23.6049 R:17.0000 rate:0.0340 aloss:0.6870 eloss:3.0503 aloss2:0.7021 exploreP:0.8277
Episode:81 meanR:23.6220 R:25.0000 rate:0.0500 aloss:0.7073 eloss:3.0598 aloss2:0.7016 exploreP:0.8257
Episode:82 meanR:24.0120 R:56.0000 rate:0.1120 aloss:0.7017 eloss:3.0653 aloss2:0.7025 exploreP:0.8211
Episode:83 meanR:24.0119 R:24.0000 rate:0.0480 aloss:0.6859 eloss:3.0403 aloss2:0.7037 exploreP:0.8192
Episode:84 meanR:24.1176 R:33.0000 rate:0.0660 aloss:0.6936 eloss:3.0472 aloss2:0.7046 exploreP:0.8165
Episode:85 meanR:24.0465 R:18.0000 rate:0.0360 aloss:0.7060 eloss:3.0458 aloss2:0.7046 exploreP:0.8151
Episode:86 meanR:24.1954 R:37.0000 rate:0.0740 aloss:0.7077 eloss:3.0621 aloss2:0.7024 exploreP:0.8121
Episode:87 meanR:24.1932 R:24.0000 rate:0.0480 aloss:0.7092 eloss:3.0569 aloss2:0.7058 exploreP:0.8102
Episode:88 meanR:24.4157 R:44.0000 rate:0.0880 aloss:0.7185 eloss:3.0598 aloss2:0.7046 exploreP:0.8066
Episode:89 meanR:24.6667 R:47.0000 rate:0.0940 aloss:0.7091 eloss:3.0541 

Episode:159 meanR:43.5400 R:42.0000 rate:0.0840 aloss:0.7510 eloss:2.0197 aloss2:1.3833 exploreP:0.5657
Episode:160 meanR:43.9100 R:52.0000 rate:0.1040 aloss:0.7770 eloss:1.9700 aloss2:1.4211 exploreP:0.5628
Episode:161 meanR:44.0600 R:40.0000 rate:0.0800 aloss:0.7713 eloss:1.9146 aloss2:1.4781 exploreP:0.5606
Episode:162 meanR:44.1500 R:32.0000 rate:0.0640 aloss:0.8081 eloss:1.8845 aloss2:1.5019 exploreP:0.5588
Episode:163 meanR:44.6100 R:63.0000 rate:0.1260 aloss:0.7537 eloss:1.8154 aloss2:1.5636 exploreP:0.5554
Episode:164 meanR:44.8200 R:42.0000 rate:0.0840 aloss:0.8030 eloss:1.8230 aloss2:1.5862 exploreP:0.5531
Episode:165 meanR:45.2500 R:57.0000 rate:0.1140 aloss:0.7460 eloss:1.7422 aloss2:1.6836 exploreP:0.5500
Episode:166 meanR:45.7600 R:68.0000 rate:0.1360 aloss:0.8008 eloss:1.7198 aloss2:1.6972 exploreP:0.5464
Episode:167 meanR:45.2500 R:12.0000 rate:0.0240 aloss:0.8768 eloss:1.7024 aloss2:1.7285 exploreP:0.5457
Episode:168 meanR:45.8300 R:74.0000 rate:0.1480 aloss:0.7557 elo

Episode:238 meanR:33.5600 R:14.0000 rate:0.0280 aloss:0.7406 eloss:1.4243 aloss2:2.4896 exploreP:0.4555
Episode:239 meanR:33.4400 R:11.0000 rate:0.0220 aloss:0.8230 eloss:1.4330 aloss2:2.4145 exploreP:0.4551
Episode:240 meanR:33.0300 R:11.0000 rate:0.0220 aloss:0.7800 eloss:1.6938 aloss2:2.3919 exploreP:0.4546
Episode:241 meanR:32.6800 R:11.0000 rate:0.0220 aloss:0.7204 eloss:1.4779 aloss2:2.4091 exploreP:0.4541
Episode:242 meanR:32.0000 R:10.0000 rate:0.0200 aloss:0.7504 eloss:1.3965 aloss2:2.4812 exploreP:0.4536
Episode:243 meanR:31.6100 R:11.0000 rate:0.0220 aloss:0.7737 eloss:1.4278 aloss2:2.5016 exploreP:0.4531
Episode:244 meanR:30.0600 R:14.0000 rate:0.0280 aloss:0.7502 eloss:1.4364 aloss2:2.5056 exploreP:0.4525
Episode:245 meanR:29.3700 R:12.0000 rate:0.0240 aloss:0.7651 eloss:1.4120 aloss2:2.4294 exploreP:0.4520
Episode:246 meanR:29.1000 R:10.0000 rate:0.0200 aloss:0.7498 eloss:1.4535 aloss2:2.5429 exploreP:0.4516
Episode:247 meanR:28.8100 R:13.0000 rate:0.0260 aloss:0.7705 elo

Episode:317 meanR:13.4200 R:14.0000 rate:0.0280 aloss:0.7720 eloss:1.4829 aloss2:2.5594 exploreP:0.4128
Episode:318 meanR:13.1800 R:46.0000 rate:0.0920 aloss:0.7143 eloss:1.3950 aloss2:2.5450 exploreP:0.4109
Episode:319 meanR:13.1800 R:14.0000 rate:0.0280 aloss:0.7451 eloss:1.3920 aloss2:2.3956 exploreP:0.4104
Episode:320 meanR:13.1800 R:12.0000 rate:0.0240 aloss:0.6984 eloss:1.4238 aloss2:2.7041 exploreP:0.4099
Episode:321 meanR:13.3700 R:42.0000 rate:0.0840 aloss:0.6997 eloss:1.3742 aloss2:2.5149 exploreP:0.4082
Episode:322 meanR:13.6100 R:40.0000 rate:0.0800 aloss:0.7115 eloss:1.3837 aloss2:2.4858 exploreP:0.4066
Episode:323 meanR:13.9200 R:43.0000 rate:0.0860 aloss:0.7215 eloss:1.3734 aloss2:2.4792 exploreP:0.4049
Episode:324 meanR:14.3000 R:47.0000 rate:0.0940 aloss:0.6974 eloss:1.3834 aloss2:2.5332 exploreP:0.4031
Episode:325 meanR:15.0700 R:85.0000 rate:0.1700 aloss:0.7088 eloss:1.4226 aloss2:2.5268 exploreP:0.3997
Episode:326 meanR:15.3500 R:39.0000 rate:0.0780 aloss:0.7023 elo

...

Episode:1030 meanR:10.2800 R:9.0000 rate:0.0180 aloss:0.7139 eloss:1.3007 aloss2:3.0136 exploreP:0.1299
Episode:1031 meanR:10.2900 R:12.0000 rate:0.0240 aloss:0.7215 eloss:1.2878 aloss2:2.8146 exploreP:0.1297
Episode:1032 meanR:10.2800 R:9.0000 rate:0.0180 aloss:0.7285 eloss:1.3876 aloss2:2.9873 exploreP:0.1296
Episode:1033 meanR:10.2800 R:10.0000 rate:0.0200 aloss:0.7132 eloss:1.2716 aloss2:2.6344 exploreP:0.1295
Episode:1034 meanR:10.2900 R:10.0000 rate:0.0200 aloss:0.7019 eloss:1.3317 aloss2:2.5798 exploreP:0.1294
Episode:1035 meanR:10.2900 R:11.0000 rate:0.0220 aloss:0.7218 eloss:1.3103 aloss2:2.7515 exploreP:0.1292
Episode:1036 meanR:10.2800 R:8.0000 rate:0.0160 aloss:0.6676 eloss:1.6076 aloss2:2.6637 exploreP:0.1291
Episode:1037 meanR:10.2900 R:10.0000 rate:0.0200 aloss:0.7068 eloss:1.2855 aloss2:2.7205 exploreP:0.1290
Episode:1038 meanR:10.2600 R:10.0000 rate:0.0200 aloss:0.7183 eloss:1.3712 aloss2:2.8385 exploreP:0.1289
Episode:1039 meanR:10.2400 R:11.0000 rate:0.0220 aloss:0.6

Episode:1109 meanR:10.2200 R:10.0000 rate:0.0200 aloss:0.7377 eloss:1.2840 aloss2:2.7574 exploreP:0.1207
Episode:1110 meanR:10.2100 R:10.0000 rate:0.0200 aloss:0.7497 eloss:1.3085 aloss2:2.8533 exploreP:0.1206
Episode:1111 meanR:10.1700 R:10.0000 rate:0.0200 aloss:0.7369 eloss:1.3049 aloss2:2.9411 exploreP:0.1204
Episode:1112 meanR:10.1600 R:10.0000 rate:0.0200 aloss:0.7199 eloss:1.3009 aloss2:2.8207 exploreP:0.1203
Episode:1113 meanR:10.1600 R:9.0000 rate:0.0180 aloss:0.7445 eloss:1.3043 aloss2:2.8204 exploreP:0.1202
Episode:1114 meanR:10.1600 R:9.0000 rate:0.0180 aloss:0.7469 eloss:1.2652 aloss2:2.8071 exploreP:0.1201
Episode:1115 meanR:10.1800 R:12.0000 rate:0.0240 aloss:0.7000 eloss:1.2983 aloss2:2.6904 exploreP:0.1200
Episode:1116 meanR:10.1300 R:9.0000 rate:0.0180 aloss:0.7177 eloss:1.3607 aloss2:2.7849 exploreP:0.1199
Episode:1117 meanR:10.1400 R:12.0000 rate:0.0240 aloss:0.7560 eloss:1.2857 aloss2:2.8123 exploreP:0.1198
Episode:1118 meanR:10.1200 R:8.0000 rate:0.0160 aloss:0.73

Episode:1188 meanR:10.1800 R:10.0000 rate:0.0200 aloss:0.7656 eloss:1.3297 aloss2:2.8541 exploreP:0.1121
Episode:1189 meanR:10.1700 R:8.0000 rate:0.0160 aloss:0.7433 eloss:1.3190 aloss2:2.7885 exploreP:0.1120
Episode:1190 meanR:10.1800 R:9.0000 rate:0.0180 aloss:0.7308 eloss:1.3278 aloss2:2.7349 exploreP:0.1119
Episode:1191 meanR:10.1900 R:11.0000 rate:0.0220 aloss:0.7444 eloss:1.2931 aloss2:2.7492 exploreP:0.1118
Episode:1192 meanR:10.1800 R:8.0000 rate:0.0160 aloss:0.7377 eloss:1.2996 aloss2:2.8680 exploreP:0.1117
Episode:1193 meanR:10.1900 R:10.0000 rate:0.0200 aloss:0.7595 eloss:1.3301 aloss2:2.9133 exploreP:0.1116
Episode:1194 meanR:10.1400 R:8.0000 rate:0.0160 aloss:0.7187 eloss:1.2686 aloss2:2.7554 exploreP:0.1116
Episode:1195 meanR:10.1500 R:10.0000 rate:0.0200 aloss:0.7191 eloss:1.2656 aloss2:2.7775 exploreP:0.1115
Episode:1196 meanR:10.1300 R:8.0000 rate:0.0160 aloss:0.7235 eloss:1.2594 aloss2:2.7349 exploreP:0.1114
Episode:1197 meanR:10.1200 R:9.0000 rate:0.0180 aloss:0.7227

Episode:1267 meanR:10.1500 R:10.0000 rate:0.0200 aloss:0.7760 eloss:1.4012 aloss2:2.8996 exploreP:0.1042
Episode:1268 meanR:10.2000 R:13.0000 rate:0.0260 aloss:0.7657 eloss:1.3692 aloss2:2.8785 exploreP:0.1040
Episode:1269 meanR:10.2100 R:10.0000 rate:0.0200 aloss:0.7198 eloss:1.2607 aloss2:2.8248 exploreP:0.1039
Episode:1270 meanR:10.2000 R:9.0000 rate:0.0180 aloss:0.7362 eloss:1.2852 aloss2:2.6855 exploreP:0.1039
Episode:1271 meanR:10.1800 R:10.0000 rate:0.0200 aloss:0.6896 eloss:1.2682 aloss2:2.7575 exploreP:0.1038
Episode:1272 meanR:10.1800 R:10.0000 rate:0.0200 aloss:0.7306 eloss:1.2552 aloss2:2.7065 exploreP:0.1037
Episode:1273 meanR:10.1700 R:10.0000 rate:0.0200 aloss:0.7786 eloss:1.3113 aloss2:2.9050 exploreP:0.1036
Episode:1274 meanR:10.1700 R:9.0000 rate:0.0180 aloss:0.6978 eloss:1.4068 aloss2:2.6978 exploreP:0.1035
Episode:1275 meanR:10.1900 R:10.0000 rate:0.0200 aloss:0.7375 eloss:1.3452 aloss2:2.7925 exploreP:0.1034
Episode:1276 meanR:10.1800 R:9.0000 rate:0.0180 aloss:0.7

Episode:1346 meanR:10.1200 R:10.0000 rate:0.0200 aloss:0.7715 eloss:1.3057 aloss2:2.8434 exploreP:0.0969
Episode:1347 meanR:10.1400 R:10.0000 rate:0.0200 aloss:0.7668 eloss:1.2672 aloss2:2.8536 exploreP:0.0968
Episode:1348 meanR:10.1200 R:10.0000 rate:0.0200 aloss:0.7824 eloss:1.3012 aloss2:2.8016 exploreP:0.0967
Episode:1349 meanR:10.1100 R:8.0000 rate:0.0160 aloss:0.7974 eloss:1.3030 aloss2:2.8493 exploreP:0.0966
Episode:1350 meanR:10.1100 R:10.0000 rate:0.0200 aloss:0.7956 eloss:1.3307 aloss2:2.9370 exploreP:0.0965
Episode:1351 meanR:10.0900 R:8.0000 rate:0.0160 aloss:0.7803 eloss:1.3436 aloss2:2.6868 exploreP:0.0965
Episode:1352 meanR:10.1200 R:13.0000 rate:0.0260 aloss:0.7431 eloss:1.2761 aloss2:2.8474 exploreP:0.0964
Episode:1353 meanR:10.1300 R:10.0000 rate:0.0200 aloss:0.7310 eloss:1.2770 aloss2:2.7241 exploreP:0.0963
Episode:1354 meanR:10.1300 R:10.0000 rate:0.0200 aloss:0.7756 eloss:1.3154 aloss2:2.8895 exploreP:0.0962
Episode:1355 meanR:10.1300 R:9.0000 rate:0.0180 aloss:0.7

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [21]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
